<a href="https://colab.research.google.com/github/NathaliL/Recidivism-Prediction/blob/main/Modified_COMPAS_Recidivism_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align="center">BERT tutorial: Classify Recidivism using COMPAS dataset</h2>

In [1]:
!pip install transformers scikit-learn datasets

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset
from sklearn.model_selection import KFold


<h4>Load and preprocess the dataset from Hugging Face</h4>

In [3]:
# Load and inspect the dataset from Hugging Face
dataset = load_dataset('imodels/compas-recidivism')
df = dataset['train'].to_pandas()

# Display the first few rows to check the column names
print(df.head())
print(df.columns)



texts = df['age'].astype(str).tolist()  #  text column name
labels = df['is_recid']  #  'is_recid' is the column containing labels

print(texts[:5])
print(labels[:5])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4937 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1235 [00:00<?, ? examples/s]

    age  priors_count  days_b_screening_arrest  c_jail_time  juv_fel_count  \
0  40.0           0.0                     -1.0          0.0            0.0   
1  25.0           1.0                     -1.0          1.0            0.0   
2  36.0          11.0                     -1.0          2.0            0.0   
3  23.0           1.0                     -1.0          0.0            0.0   
4  29.0           0.0                      0.0         -1.0            0.0   

   juv_other_count  juv_misd_count  c_charge_degree:F  c_charge_degree:M  \
0              0.0             0.0                0.0                1.0   
1              0.0             0.0                1.0                0.0   
2              0.0             0.0                0.0                1.0   
3              0.0             1.0                0.0                1.0   
4              0.0             0.0                0.0                1.0   

   race:African-American  ...  race:Caucasian  race:Hispanic  \
0         

In [4]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
def get_embeddings(text_list):
    embeddings = []
    for text in text_list:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings.append(outputs.last_hidden_state[:, 0, :].cpu().numpy())
    return np.vstack(embeddings)

<h4>Generate embeddings for the entire dataset</h4>

In [6]:
embeddings = get_embeddings(texts)

<h4>Split the data into training and test sets</h4>

In [7]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

<h4>Train the Logistic Regression classifier</h4>

In [8]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

<h4>Predict and evaluate the classifier</h4>

In [20]:
#y_pred = clf.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)
#precision = precision_score(y_test, y_pred, average='binary')
#recall = recall_score(y_test, y_pred, average='binary')
#f1 = f1_score(y_test, y_pred, average='binary')

#print(f"Accuracy: {accuracy}")
#print(f"Precision: {precision}")
#print(f"Recall: {recall}")
#print(f"F1 Score: {f1}")

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)
Y = labels  # Labels: 'is_recid' column


def perform_k_fold_cv(model, X, y, k):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model.fit(X_train, y_train)
        predictions = model.predict(X_val)

        accuracy_scores.append(accuracy_score(y_val, predictions))
        precision_scores.append(precision_score(y_val, predictions, average='binary'))
        recall_scores.append(recall_score(y_val, predictions, average='binary'))
        f1_scores.append(f1_score(y_val, predictions, average='binary'))

    return accuracy_scores, precision_scores, recall_scores, f1_scores

    model = LogisticRegression(max_iter=1000)

acc_k5, prec_k5, recall_k5, f1_k5 = perform_k_fold_cv(model, X, y, 5)
acc_k10, prec_k10, recall_k10, f1_k10 = perform_k_fold_cv(model, X, y, 10)

print("K=5 Accuracy:", acc_k5)
print("K=5 Precision:", prec_k5)
print("K=5 Recall:", recall_k5)
print("K=5 F1 Score:", f1_k5)

print("K=10 Accuracy:", acc_k10)
print("K=10 Precision:", prec_k10)
print("K=10 Recall:", recall_k10)
print("K=10 F1 Score:", f1_k10)


K=5 Accuracy: [0.597165991902834, 0.5779352226720648, 0.5612968591691996, 0.5785207700101317, 0.563323201621074]
K=5 Precision: [0.568760611205433, 0.5868544600938967, 0.5373406193078324, 0.5647668393782384, 0.5351351351351351]
K=5 Recall: [0.6993736951983298, 0.5091649694501018, 0.6223628691983122, 0.6659877800407332, 0.6319148936170212]
K=5 F1 Score: [0.6273408239700375, 0.54525627044711, 0.5767350928641252, 0.611214953271028, 0.5795121951219512]
K=10 Accuracy: [0.6052631578947368, 0.5890688259109311, 0.6153846153846154, 0.5910931174089069, 0.5506072874493927, 0.5728744939271255, 0.5890688259109311, 0.5679513184584178, 0.5578093306288032, 0.5578093306288032]
K=10 Precision: [0.5719063545150501, 0.5655172413793104, 0.5890909090909091, 0.5850340136054422, 0.5364963503649635, 0.5358361774744027, 0.5870307167235495, 0.5435540069686411, 0.5038167938931297, 0.5543859649122806]
K=10 Recall: [0.7184873949579832, 0.6804979253112033, 0.6778242677824268, 0.6825396825396826, 0.6074380165289256, 